In [1]:
import multiprocessing
import os
from pathlib import Path

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec

In [2]:
pd.set_option('display.max_colwidth', None)

main_path = Path.cwd() / ".."
songs_list = os.listdir(main_path / "data/txt")

Vamos a utilizar como corpus la letra de las cacniones de Atahualpa Yupanqui, descargadas del sitio web [letras.com](https://www.letras.com/atahualpa-yupanqui/), utilizando la tecnica de web scraping.

Como documentos seleccionamos cada uno de los parrafos dentro de las canciones, esto es, estrofas y estribillos. Esto debido a que, por lo general, en las canciones de folclore cada parrafo se encuentra contextualizado en si mismo. Ademas, si tomaramos cada verso como corpus encontrariamos que algunas canciones poseen versos de tan solo algunas palabras.

In [3]:
# load the documents.
list_dfs = []
for song in songs_list:
    try:
        list_dfs.append(pd.read_csv(main_path / f"data/txt/{song}", sep='/n', header=None, engine='python'))
    except Exception as e:
        print(f"Error: couldn't load {song}")

df = pd.concat(list_dfs).rename(columns = {0: "lyrics"})

Error: couldn't load Cancion Del Abuelo N 2.txt
Error: couldn't load Cuando Ya Nadie Te Nombre.txt
Error: couldn't load Danza de La Luna.txt
Error: couldn't load Danza Santiaguena.txt
Error: couldn't load El Mal Dormido.txt
Error: couldn't load El Rescoldeao.txt
Error: couldn't load Estilo Serrano.txt
Error: couldn't load Gato Santiagueno.txt
Error: couldn't load La Estancia Vieja.txt
Error: couldn't load La Nadita.txt
Error: couldn't load Malquistao o Vidala Dolorosa.txt
Error: couldn't load Melodia Del Adios y Danza Rustica.txt
Error: couldn't load Milonga triste.txt
Error: couldn't load Paso de Los Andes.txt
Error: couldn't load Vidala.txt
Error: couldn't load Zamba Tucumana.txt


In [4]:
print(f"Cantidad de canciones en el corpus: {len(list_dfs)}")

Cantidad de canciones en el corpus: 2026


In [5]:
# Visualize the first song
list_dfs[0].head(5)

,0
0,Estaba el Cerro tranquilo Cada cual en su trabajo Cuando llegaron de abajo Tres democratas eternos Emisarios del Gobierno La lengua como badajo
1,Reunir a la paisanada Traian como mision Para hacer una funcion De asados y de empanadas Y asi tomar posesion De toda gruta pintada
2,"Se busco primeramente La sombra de Arganaras Pero dada la humedad Y del mucho genterio En los talas, junto al rio Se vido mas ampliedad"
3,"Para sacar el yuyal Con hachas, palas y picos Trajeron muchos melicos De los tres Departamentos Y a los pobres, al momento Les humeaban los hocicos"
4,Lo que antes fue matorral Quedo mesmo que un salon Regadito y parejon Y medio atras del ramaje Un bano pa'l mujeraje Con forma de corazon


In [6]:
print(f"Cantidad de documentos en el corpus: {df.shape[0]}")

Cantidad de documentos en el corpus: 13372


In [7]:
df["lyrics"] = df["lyrics"].replace('"', '')

df.to_csv(main_path / f"data/songs.csv", index=False)

In [8]:
df

,lyrics
0,Estaba el Cerro tranquilo Cada cual en su trabajo Cuando llegaron de abajo Tres democratas eternos Emisarios del Gobierno La lengua como badajo
1,Reunir a la paisanada Traian como mision Para hacer una funcion De asados y de empanadas Y asi tomar posesion De toda gruta pintada
2,"Se busco primeramente La sombra de Arganaras Pero dada la humedad Y del mucho genterio En los talas, junto al rio Se vido mas ampliedad"
3,"Para sacar el yuyal Con hachas, palas y picos Trajeron muchos melicos De los tres Departamentos Y a los pobres, al momento Les humeaban los hocicos"
4,Lo que antes fue matorral Quedo mesmo que un salon Regadito y parejon Y medio atras del ramaje Un bano pa'l mujeraje Con forma de corazon
...,...
4,"Aquel que trabaja el campo con alma quiero cantar, y aunque a veces ignoramos lo que es la zona rural, si no fuera por su esfuerzo no habria esperanza que contar."
5,"Son meses lejos de casa, de trabajar sin parar, durmiendo en una casilla de comer hasta que hay un perro de compania vagabundo del lugar."
6,"Despacito y con paciencia viajando para el lugar las maquinas sin apuro con el equipo detras, controles de policia de noche no hay que continuar."
7,"Despacito y con paciencia volver a casa sera, reencuentro con la familia y volver a descansar estar en casa de nuevo no se puede comparar."


#### 1. Preprocesamiento

In [32]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [35]:
# Demos un vistazo
sentence_tokens[0]

['estaba',
 'el',
 'cerro',
 'tranquilo',
 'cada',
 'cual',
 'en',
 'su',
 'trabajo',
 'cuando',
 'llegaron',
 'de',
 'abajo',
 'tres',
 'democratas',
 'eternos',
 'emisarios',
 'del',
 'gobierno',
 'la',
 'lengua',
 'como',
 'badajo']

In [128]:
import re

# Remove stopwords
stopwords_es = nltk.corpus.stopwords.words("spanish")
sentence_tokens_nostop = [
    [
        re.sub(r"[^a-zA-Z0-9 ]", "", word)
        for word in document
        if word not in stopwords_es
    ]
    for document in sentence_tokens
]

# Remove words with less than 3 letters
sentence_tokens_nostop = [
    [
        word
        for word in document
        if len(word) >= 3
    ]
    for document in sentence_tokens_nostop
]

#### 2. Crear los vectores Word2Vec

In [129]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [218]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=7,       # cant de palabras antes y desp de la predicha
                     vector_size=32,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [219]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens_nostop)

In [220]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2187


In [221]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1024
